# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abdullahghifari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/abdullahghifari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abdullahghifari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/abdullahghifari/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/disaster.db')
df = pd.read_sql_table('messages_clean_table',con=engine)
print(df.shape)

(26216, 40)


In [4]:
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
vectorizer = TfidfVectorizer( tokenizer=tokenize ,use_idf=True, smooth_idf=True, sublinear_tf=False)

rf = RandomForestClassifier(random_state=123)

pipeline = Pipeline([
    ('vectorizer',vectorizer),
    ('model',rf)
])# compute bag of word counts and tf-idf values

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=123)
pipeline.fit(X_train,y_train)

/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x16182b760>)),
                ('model', RandomForestClassifier(random_state=123))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

In [9]:
def get_score(y_true,y_pred):
    """Get Score for Each Column"""
    
    accuracy = accuracy_score(y_true,y_pred)
    precision =round( precision_score(y_true,y_pred,average='micro'))
    recall = recall_score(y_true,y_pred,average='micro')
    f1 = f1_score(y_true,y_pred,average='micro')
    
    return {'Precision':precision, 'Recall':recall,'F1-score':f1,'Accuracy':accuracy}

In [10]:
train_results = []
for i,column in enumerate(y_train.columns):
    result = get_score(y_train.loc[:,column].values,y_train_pred[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df    

,Precision,Recall,F1-score,Accuracy
0,1,0.997143,0.997143,0.997143
1,1,1.000000,1.000000,1.000000
2,1,0.999286,0.999286,0.999286
3,1,0.998571,0.998571,0.998571
4,1,0.999286,0.999286,0.999286
5,1,0.998571,0.998571,0.998571
6,1,1.000000,1.000000,1.000000
7,1,1.000000,1.000000,1.000000
8,1,1.000000,1.000000,1.000000
9,1,1.000000,1.000000,1.000000


In [11]:
test_results = []
for i,column in enumerate(y_test.columns):
    result = get_score(y_test.loc[:,column].values,y_test_pred[:,i])
    test_results.append(result)
test_results_df = pd.DataFrame(test_results)
test_results_df

,Precision,Recall,F1-score,Accuracy
0,1,0.776667,0.776667,0.776667
1,1,0.898333,0.898333,0.898333
2,1,0.998333,0.998333,0.998333
3,1,0.716667,0.716667,0.716667
4,1,0.915000,0.915000,0.915000
5,1,0.951667,0.951667,0.951667
6,1,0.966667,0.966667,0.966667
7,1,0.976667,0.976667,0.976667
8,1,0.965000,0.965000,0.965000
9,1,1.000000,1.000000,1.000000


In [12]:
train_results_df['F1-score'].mean()

0.9997420634920634

In [13]:
test_results_df['F1-score'].mean()

0.9403240740740743

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
def custom_multiclf_score(y_true,y_pred):
    list_results = []
    for i,column in enumerate(y_true.columns):
        result = f1_score(y_true.loc[:,column].values,y_pred[:,i],average='micro')
        list_results.append(result)
    list_results = np.array(list_results)
    return list_results.mean()

In [15]:
from sklearn.metrics import make_scorer
score = make_scorer(custom_multiclf_score)

In [16]:
parameters = {
    'model__max_depth':[2,4,None],
    'model__max_features':['sqrt','log2'],
}

cv = GridSearchCV(
    estimator=pipeline,
    param_grid=parameters,
    scoring=score,
#     n_jobs=-1, 
    cv=5, 
    refit=True, 
    return_train_score=True,
    verbose=10
)

In [17]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START model__max_depth=2, model__max_features=sqrt................


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/5; 1/6] END model__max_depth=2, model__max_features=sqrt;, score=(train=0.927, test=0.929) total time=   2.2s
[CV 2/5; 1/6] START model__max_depth=2, model__max_features=sqrt................


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/5; 1/6] END model__max_depth=2, model__max_features=sqrt;, score=(train=0.927, test=0.928) total time=   2.3s
[CV 3/5; 1/6] START model__max_depth=2, model__max_features=sqrt................


...

[CV 4/5; 6/6] END model__max_depth=None, model__max_features=log2;, score=(train=1.000, test=0.939) total time=   3.5s
[CV 5/5; 6/6] START model__max_depth=None, model__max_features=log2.............


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 5/5; 6/6] END model__max_depth=None, model__max_features=log2;, score=(train=1.000, test=0.933) total time=   2.9s


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x16182b760>)),
                                       ('model',
                                        RandomForestClassifier(random_state=123))]),
             param_grid={'model__max_depth': [2, 4, None],
                         'model__max_features': ['sqrt', 'log2']},
             return_train_score=True,
             scoring=make_scorer(custom_multiclf_score), verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
y_train_pred = cv.predict(X_train)
y_test_pred = cv.predict(X_test)

In [19]:
train_results = []
for i,column in enumerate(y_train.columns):
    result = get_score(y_train.loc[:,column].values,y_train_pred[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df    

,Precision,Recall,F1-score,Accuracy
0,1,0.997143,0.997143,0.997143
1,1,1.000000,1.000000,1.000000
2,1,0.999286,0.999286,0.999286
3,1,0.998571,0.998571,0.998571
4,1,0.999286,0.999286,0.999286
5,1,0.998571,0.998571,0.998571
6,1,1.000000,1.000000,1.000000
7,1,1.000000,1.000000,1.000000
8,1,1.000000,1.000000,1.000000
9,1,1.000000,1.000000,1.000000


In [20]:
test_results = []
for i,column in enumerate(y_test.columns):
    result = get_score(y_test.loc[:,column].values,y_test_pred[:,i])
    test_results.append(result)
test_results_df = pd.DataFrame(test_results)
test_results_df

,Precision,Recall,F1-score,Accuracy
0,1,0.776667,0.776667,0.776667
1,1,0.898333,0.898333,0.898333
2,1,0.998333,0.998333,0.998333
3,1,0.716667,0.716667,0.716667
4,1,0.915000,0.915000,0.915000
5,1,0.951667,0.951667,0.951667
6,1,0.966667,0.966667,0.966667
7,1,0.976667,0.976667,0.976667
8,1,0.965000,0.965000,0.965000
9,1,1.000000,1.000000,1.000000


In [21]:
train_results_df['F1-score'].mean()

0.9997420634920634

In [22]:
test_results_df['F1-score'].mean()

0.9403240740740743

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
vectorizer = CountVectorizer( tokenizer=tokenize)

model = KNeighborsClassifier()

pipeline = Pipeline([
    ('vectorizer',vectorizer),
    ('model',model)
])

In [24]:
parameters = {
    'model__n_neighbors':[1,2,3,4,5]
}

In [25]:
cv_nb = GridSearchCV(
    pipeline,
    param_grid=parameters, 
    scoring=score,
#     n_jobs = -1, 
    cv = 5, 
    refit = True, 
    return_train_score=True,
    verbose=10)

cv_nb.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5; 1/5] START model__n_neighbors=1........................................


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/5; 1/5] END model__n_neighbors=1;, score=(train=1.000, test=0.916) total time=   1.9s
[CV 2/5; 1/5] START model__n_neighbors=1........................................


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/5; 1/5] END model__n_neighbors=1;, score=(train=1.000, test=0.913) total time=   1.9s
[CV 3/5; 1/5] START model__n_neighbors=1........................................


...

[CV 4/5; 5/5] END model__n_neighbors=5;, score=(train=0.942, test=0.931) total time=   1.9s
[CV 5/5; 5/5] START model__n_neighbors=5........................................


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 5/5; 5/5] END model__n_neighbors=5;, score=(train=0.937, test=0.923) total time=   1.9s


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        CountVectorizer(tokenizer=<function tokenize at 0x16182b760>)),
                                       ('model', KNeighborsClassifier())]),
             param_grid={'model__n_neighbors': [1, 2, 3, 4, 5]},
             return_train_score=True,
             scoring=make_scorer(custom_multiclf_score), verbose=10)

In [26]:
y_train_pred = cv_nb.predict(X_train)
y_test_pred = cv_nb.predict(X_test)

In [27]:
train_results = []
for i,column in enumerate(y_train.columns):
    result = get_score(y_train.loc[:,column].values,y_train_pred[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df    

,Precision,Recall,F1-score,Accuracy
0,1,0.897143,0.897143,0.897143
1,1,0.877143,0.877143,0.877143
2,1,0.992857,0.992857,0.992857
3,1,0.729286,0.729286,0.729286
4,1,0.928571,0.928571,0.928571
5,1,0.954286,0.954286,0.954286
6,1,0.976429,0.976429,0.976429
7,1,0.981429,0.981429,0.981429
8,1,0.969286,0.969286,0.969286
9,1,1.000000,1.000000,1.000000


In [28]:
test_results = []
for i,column in enumerate(y_test.columns):
    result = get_score(y_test.loc[:,column].values,y_test_pred[:,i])
    test_results.append(result)
test_results_df = pd.DataFrame(test_results)
test_results_df

,Precision,Recall,F1-score,Accuracy
0,1,0.650000,0.650000,0.650000
1,1,0.870000,0.870000,0.870000
2,1,0.998333,0.998333,0.998333
3,1,0.651667,0.651667,0.651667
4,1,0.915000,0.915000,0.915000
5,1,0.951667,0.951667,0.951667
6,1,0.965000,0.965000,0.965000
7,1,0.976667,0.976667,0.976667
8,1,0.965000,0.965000,0.965000
9,1,1.000000,1.000000,1.000000


In [29]:
train_results_df['F1-score'].mean()

0.9462698412698415

In [30]:
test_results_df['F1-score'].mean()

0.9291203703703703

### 9. Export your model as a pickle file

In [31]:
import pickle

filename = 'best_model_rf_tuned.pkl'
pickle.dump(cv, open(filename, 'wb'))

filename = 'best_model_nb.pkl'
pickle.dump(cv_nb, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.